In [1]:
import JupyterNotebooksLib as slicernb
import slicer
import os 
from DICOMLib import DICOMUtils
import numpy as np
import zipfile

from glob import glob
from natsort import natsorted
from tqdm import tqdm

# Set image viewer size to 50% screen size
# slicernb.AppWindow.setWindowSize(scale=0.5)
# slicernb.AppWindow.setContents("viewers")

# Clear scene
slicer.mrmlScene.Clear(False)

# Unzip and organize files 

In [2]:
#input folder 
input_folder = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\Segmentation database"
intermediary_folder = "./temp_unzip"

if  not os.path.exists(intermediary_folder):
    os.mkdir(intermediary_folder)

In [3]:
for zip_file in os.listdir(input_folder):
    if "(" in zip_file or ")" in zip_file:
        continue;
    zip_path = f"{input_folder}/{zip_file}"
    Case_id = zip_file.split("_")[0]

    if "_segmented_structures" in zip_file: 
        out_path = f"{intermediary_folder}/{Case_id}_segmentations"
    else:
        out_path = f"{intermediary_folder}/{Case_id}_volumes"
        
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(out_path)

# organize file names throw out unuasable cases 

In [4]:
input_cases = [] 


volumes_paths = list(natsorted(glob(os.path.join(intermediary_folder,  "*"+ f"_volumes"))))

for vol in volumes_paths: 

    seg_path = vol.replace ("_volumes","_segmentations")

    if (os.path.exists(seg_path)):
        input_cases.append ({"vol":vol,"seg":seg_path})
    else:
        print ("cannot find matching segmentation case will be skipped")



cannot find matching segmentation case will be skipped
cannot find matching segmentation case will be skipped


In [5]:
input_cases[0]["vol"].split("\\")[-1].split("_")[0]

'304846'

# using slicer to convert to nii

### will need to alter segmentation nodes to reflect changes in classes you are trying to segment

In [6]:
nii_path = "./nii_ds_2"
nii_path_volumes = f"{nii_path}/Volumes"
nii_path_segmentations = f"{nii_path}/Masks"

try:
    os.mkdir(nii_path)
    os.mkdir(nii_path_volumes)
    os.mkdir(nii_path_segmentations)
except:
    print ("these directories already exist")

In [7]:
def get_model_class_from_file (file): 
    if "tooth" in file: 
        #print (file.split("tooth_")[1].split("_")[0])

        if (".stl" in file):
            return "tooth_" + file.split("tooth_")[1].split("_")[0]
        else:
            return "tooth_" + file.split("tooth-")[1].split("-")[0].split("_")[0]

    else: 
        # CaseId = file.split("_") [-1].split(".)
        # #print (file.split(f"{CaseId}_")[-1].split("_")[0])
        if (".stl" in file):
            return file.split("_") [-1].split(".")[0]
        else:
            return file.split("-") [-1].split("_")[0]

In [8]:
get_model_class_from_file ("573401_4172530600-cbct-tooth-31-pulp_decimated.obj")
#"304846_canal.stl".split("_")

'tooth_31'

In [9]:
lable_map = {
    "background":0,
    "sinus": 1,
    "canal": 2,
    "mandible": 3,
    "maxilla":4,
    "tooth_11":5,
    "tooth_12":6,
    "tooth_13":7,
    "tooth_14":8,
    "tooth_15":9,
    "tooth_16":10,
    "tooth_17":11,
    "tooth_18":12,
    "tooth_21":13,
    "tooth_22":14,
    "tooth_23":15,
    "tooth_24":16,
    "tooth_25":17,
    "tooth_26":18,
    "tooth_27":19,
    "tooth_28":20,
    "tooth_31":21,
    "tooth_32":22,
    "tooth_33":23,
    "tooth_34":24,
    "tooth_35":25,
    "tooth_36":26,
    "tooth_37":27,
    "tooth_38":28,
    "tooth_41":29,
    "tooth_42":30,
    "tooth_43":31,
    "tooth_44":32,
    "tooth_45":33,
    "tooth_46":34,
    "tooth_47":35,
    "tooth_48":36,
}

In [10]:
def convert_to_nnunet_lable_structure (segmentationNodes,referenceVolumeNode):

    all_segment_np = [] 
    for lable, node in segmentationNodes.items():
        name = node.GetName()
    
        segmentationLblmap  = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
        segmentationLblmap.SetName(name+"_lablemap")
        slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(node, segmentationLblmap, referenceVolumeNode)
        mask = slicer.util.arrayFromVolume(segmentationLblmap)
        mask[mask > 0] = lable_map[lable]
    
        all_segment_np.append(mask.copy())



    final_out = all_segment_np[0] 
    for i in range (1,len(all_segment_np)):
        indices  = np.where (final_out <1)
        final_out [indices] = all_segment_np[i][indices]

    segmentationLblmap  = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(next(iter(segmentationNodes.values())), segmentationLblmap, referenceVolumeNode)
    
    slicer.util.updateVolumeFromArray(segmentationLblmap, final_out)

    return segmentationLblmap

    

In [11]:
maxillary_teeth_lables = ["11","12","13","14","15","16","17","18","21","22","23","24","25","26","27","28"]
mandibular_teeth_lables = ["31","32","33","34","35","36","37","38","41","42","43","44","45","46","47","48"]

def convert_single_case_to_nii (dicom_series_path,segmentation_path):

    '''
    simply loading dicoms into a fresh scene 
    '''
    
    
    # # Clear scene
    slicer.mrmlScene.Clear(False)

    print(dicom_series_path)
    print(segmentation_path)
    
    ##load dicom 
    loadedNodeIDs = []
    with DICOMUtils.TemporaryDICOMDatabase() as db:
        DICOMUtils.importDicom(os.path.abspath(dicom_series_path), db)
        patientUIDs = db.patients()
    
        print (patientUIDs)
        for patientUID in patientUIDs:
            loadedNodeIDs.extend(DICOMUtils.loadPatientByUID(patientUID))



    '''
    loading the segmentations into appropriate substructuresz 
    
    '''
    
    
    ## creating segmentation nodes
    segmentationNodes = {} 


    ## importing stls 
    nodes = []
    node_names = []
    for seg in os.listdir(segmentation_path):
        if ((".stl" in seg or ".obj" in seg) and "pulp" not in seg ):
            mesh_path = f"{segmentation_path}/{seg}"
            class_name = get_model_class_from_file(seg)

            if class_name in lable_map:

                segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
                segmentationNode.CreateDefaultDisplayNodes()
                segmentationNodes [class_name] = segmentationNode
                
                slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), segmentationNode)

    referenceVolumeNode = slicer.mrmlScene.GetNodeByID("vtkMRMLScalarVolumeNode1")

    segmentation_lable_map = convert_to_nnunet_lable_structure(segmentationNodes,referenceVolumeNode)

    caseID = dicom_series_path.split("\\")[-1].split("_")[0]
    volume_save_path = f"{nii_path_volumes}/{caseID}_volume.nii.gz"
    seg_save_path =  f"{nii_path_segmentations}/{caseID}_seg.nii.gz"

    slicer.util.exportNode(referenceVolumeNode, volume_save_path , {"useCompression": 0})
    slicer.util.exportNode(segmentation_lable_map, seg_save_path , {"useCompression": 0})

    print ("one case sucessfully converted")
     


        

In [12]:
def batch_nii_convert (in_case_files):

    for case in tqdm(in_case_files):
        try:
            convert_single_case_to_nii (case["vol"],case["seg"])
        except:
            print (f"somthing is wrong with case {case} probably a missing segmentation")
    print ("done converting all cases to nii")
    

In [13]:
batch_nii_convert (input_cases)

  0%|          | 0/238 [00:00<?, ?it/s]

./temp_unzip\304846_volumes
./temp_unzip\304846_segmentations
('1',)
one case sucessfully converted


  0%|          | 1/238 [03:47<14:58:26, 227.45s/it]

./temp_unzip\315920_volumes
./temp_unzip\315920_segmentations
('1',)
one case sucessfully converted


  1%|          | 2/238 [05:51<10:54:19, 166.35s/it]

...

./temp_unzip\579057_volumes
./temp_unzip\579057_segmentations
('1',)
one case sucessfully converted


100%|#########9| 237/238 [5:29:29<00:38, 38.81s/it]

./temp_unzip\579169_volumes
./temp_unzip\579169_segmentations
('1',)
somthing is wrong with case {'vol': './temp_unzip\\579169_volumes', 'seg': './temp_unzip\\579169_segmentations'} probably a missing segmentation


100%|##########| 238/238 [5:29:36<00:00, 83.09s/it]


done converting all cases to nii


In [14]:
# test_seg = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\304846_246033_segmented_structures"
# test_vol = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\304846_Ken_Lin_Lin_Ken____20200831_080509147"

# test_seg2 = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\temp_unzip\\576547_segmentations"
# test_vol2 = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\temp_unzip\\576547_volumes"

# # convert_single_case_to_nii (input_cases[0]["vol"],input_cases[0]["seg"])
# convert_single_case_to_nii (test_vol2,test_seg2)

# building nnunet_dataset from nii

In [15]:
#making dataset Folder 

dataset_id = 126
dataset_name = "individual_teeth" 
Dataset_folder_name = f"Dataset{dataset_id}_{dataset_name}"

if (not os.path.exists(f"./nnUNet_raw/{Dataset_folder_name}")):
    os.mkdir(f"./nnUNet_raw/{Dataset_folder_name}")

train_input_dir = f"./nnUNet_raw/{Dataset_folder_name}/imagesTr"
train_lable_dir = f"./nnUNet_raw/{Dataset_folder_name}/labelsTr"

if not os.path.exists(train_input_dir):
    os.mkdir(train_input_dir)
if not os.path.exists(train_lable_dir):
    os.mkdir(train_lable_dir)




In [16]:
import shutil
volume_list = os.listdir(nii_path_volumes)
seg_list = os.listdir(nii_path_segmentations)
# print (volume_list)

for volume in volume_list:
    volume_old_path = f"{nii_path_volumes}/{volume}"
    volume_nnunet_name = volume.replace ("_volume.nii.gz","_0000.nii.gz")
    volume_new_path = f"{train_input_dir}/{volume_nnunet_name}"

    #print (volume_nnunet_name)
    shutil.move(volume_old_path, volume_new_path)


for segmentation in seg_list:
    segmentation_old_path = f"{nii_path_segmentations}/{segmentation}"
    segmentation_nnunet_name = segmentation.replace ("_seg.nii.gz",".nii.gz")
    segmentation_new_path = f"{train_lable_dir}/{segmentation_nnunet_name}"

    #print (segmentation_nnunet_name)
    shutil.copy(segmentation_old_path, segmentation_new_path)


In [17]:
## creating Dataset Json 
import json 

dataset_metadata = {
 "channel_names": {  # formerly modalities
   "0": "CT", 
 }, 
 "labels": {
    "background":0,
    "sinus": 1,
    "canal": 2,
    "mandible": 3,
    "maxilla":4,
    "tooth_11":5,
    "tooth_12":6,
    "tooth_13":7,
    "tooth_14":8,
    "tooth_15":9,
    "tooth_16":10,
    "tooth_17":11,
    "tooth_18":12,
    "tooth_21":13,
    "tooth_22":14,
    "tooth_23":15,
    "tooth_24":16,
    "tooth_25":17,
    "tooth_26":18,
    "tooth_27":19,
    "tooth_28":20,
    "tooth_31":21,
    "tooth_32":22,
    "tooth_33":23,
    "tooth_34":24,
    "tooth_35":25,
    "tooth_36":26,
    "tooth_37":27,
    "tooth_38":28,
    "tooth_41":29,
    "tooth_42":30,
    "tooth_43":31,
    "tooth_44":32,
    "tooth_45":33,
    "tooth_46":34,
    "tooth_47":35,
    "tooth_48":36,
}, 
 "numTraining": len(volume_list), 
 "file_ending": ".nii.gz",
 "overwrite_image_reader_writer": "SimpleITKIO"  # optional! If not provided nnU-Net will automatically determine the ReaderWriter
 
}

json_data = json.dumps(dataset_metadata)

with open(f"./nnUNet_raw/{Dataset_folder_name}/dataset.json", "w") as outfile:
    outfile.write(json_data)